# 📊 Reporte de Pull Requests Abiertos en Bitbucket

Este reporte proporciona una vista completa de todos los pull requests que se encuentran abiertos en los repositorios de Bitbucket, incluyendo información detallada sobre el estado, reviewers, comentarios y tiempos de apertura para una mejor gestión y seguimiento.

## 🔧 Configuración e Importación de Librerías

In [ ]:
%pip install python-dotenv requests pandas matplotlib seaborn

import requests
import pandas as pd
from dotenv import load_dotenv
import os
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings("ignore")

print("✅ Librerías importadas correctamente")

In [ ]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Configurar credenciales de Bitbucket
username = os.getenv("BITBUCKET_USERNAME")
app_password = os.getenv("BITBUCKET_APP_PASSWORD")
workspace = os.getenv("BITBUCKET_WORKSPACE")

# Verificar que las credenciales estén configuradas
if not all([username, app_password, workspace]):
    print("❌ Error: Asegúrate de tener configuradas las variables de entorno:")
    print("- BITBUCKET_USERNAME")
    print("- BITBUCKET_APP_PASSWORD")
    print("- BITBUCKET_WORKSPACE")
else:
    print(f"✅ Credenciales configuradas correctamente para el workspace: {workspace}")
    print(f"   Usuario: {username}")

## 📦 Obtener Repositorios y Funciones de Utilidad

In [ ]:
# Función para obtener todos los elementos paginados de la API de Bitbucket
def get_all_items(url, params=None):
    """
    Obtiene todos los elementos de una URL paginada de la API de Bitbucket
    """
    items = []
    while url:
        response = requests.get(url, params=params, auth=(username, app_password))
        if response.status_code != 200:
            print(
                f"❌ Error al obtener datos: {response.status_code} - {response.text}"
            )
            break

        data = response.json()
        items.extend(data.get("values", []))

        # Obtener la siguiente página si existe
        url = data.get("next")
        params = None  # Los parámetros ya están incluidos en la URL 'next'

    return items


# Función para obtener comentarios de un pull request
def get_pr_comments(workspace, repo_slug, pr_id):
    """
    Obtiene todos los comentarios de un pull request específico
    """
    comments_url = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests/{pr_id}/comments"
    return get_all_items(comments_url)

# Función para convertir fechas a formato "para humanos" en español
def fecha_para_humanos(fecha_str):
    """
    Convierte una fecha en formato datetime o string a texto legible en español
    """
    try:
        # Si ya es string formateado, intentar parsearlo
        if isinstance(fecha_str, str):
            # Intentar varios formatos posibles
            try:
                fecha = pd.to_datetime(fecha_str, format="%Y-%m-%d %H:%M")
            except:
                fecha = pd.to_datetime(fecha_str)
        else:
            # Si es un objeto datetime de pandas
            fecha = pd.to_datetime(fecha_str)
        
        # Asegurar que la fecha tenga zona horaria UTC para comparación
        if fecha.tz is None:
            fecha = fecha.tz_localize('UTC')
        
        # Obtener tiempo actual en UTC
        ahora = pd.Timestamp.now(tz="UTC")
        
        # Calcular diferencia
        diff = ahora - fecha
        
        # Obtener componentes
        dias = diff.days
        segundos_totales = diff.total_seconds()
        horas = int(segundos_totales // 3600)
        minutos = int((segundos_totales % 3600) // 60)
        
        if dias > 0:
            if dias == 1:
                return "hace 1 día"
            elif dias < 7:
                return f"hace {dias} días"
            elif dias < 30:
                semanas = dias // 7
                if semanas == 1:
                    return "hace 1 semana"
                else:
                    return f"hace {semanas} semanas"
            elif dias < 365:
                meses = dias // 30
                if meses == 1:
                    return "hace 1 mes"
                else:
                    return f"hace {meses} meses"
            else:
                años = dias // 365
                if años == 1:
                    return "hace 1 año"
                else:
                    return f"hace {años} años"
        elif horas > 0:
            if horas == 1:
                return "hace 1 hora"
            else:
                return f"hace {horas} horas"
        elif minutos > 0:
            if minutos == 1:
                return "hace 1 minuto"
            else:
                return f"hace {minutos} minutos"
        else:
            return "hace menos de 1 minuto"
    except Exception as e:
        print(f"Error procesando fecha {fecha_str}: {e}")
        return str(fecha_str)  # Si hay error, devolver la fecha original como string

# Función para analizar comentarios del pull request según las nuevas reglas
def analyze_pr_comments(comments, pr_author_name):
    """
    Analiza los comentarios de un PR y los clasifica según las nuevas reglas (mutuamente excluyentes):
    - Devoluciones: Comentarios normales de usuarios diferentes al autor del PR (NO inline)
    - Comm_Codigo: Comentarios inline de usuarios diferentes al autor del PR
    """
    comm_pullrequest = 0
    comm_codigo = 0
    
    for comment in comments:
        # Solo procesar comentarios no eliminados
        if comment.get('deleted', False):
            continue
            
        # Obtener el nombre del usuario que hizo el comentario
        comment_author = None
        if 'user' in comment and comment['user']:
            # Priorizar display_name, luego nickname
            comment_author = comment['user'].get('display_name') or comment['user'].get('nickname')
        
        # Solo contar si el comentario es de un usuario diferente al autor del PR
        if comment_author and comment_author != pr_author_name:
            # Verificar si es comentario inline (desarrollador) o normal (pull request)
            if 'inline' in comment and comment['inline']:
                # Es un comentario inline -> comentario de código
                comm_codigo += 1
            else:
                # Es un comentario normal -> comentario del pull request
                comm_pullrequest += 1
    
    return comm_pullrequest, comm_codigo

# Función para acortar nombres largos
def acortar_nombre(nombre_completo):
    """
    Acorta nombres largos para mostrar solo primer nombre + primera letra del segundo nombre
    Ejemplo: "Daniel Felipe Leal Chaves" -> "DanielF."
    """
    if not nombre_completo or not isinstance(nombre_completo, str):
        return nombre_completo
    
    # Dividir el nombre en partes
    partes = nombre_completo.strip().split()
    
    if len(partes) == 1:
        # Solo tiene un nombre
        return partes[0]
    elif len(partes) >= 2:
        # Tiene al menos dos nombres
        primer_nombre = partes[0]
        segunda_parte = partes[1]
        
        # Devolver primer nombre + primera letra del segundo en mayúscula + punto (sin espacios)
        return f"{primer_nombre}{segunda_parte[0].upper()}."
    
    return nombre_completo


print("✅ Funciones de utilidad definidas")

In [ ]:
# Función para obtener participantes de un pull request
def get_pr_participants(workspace, repo_slug, pr_id):
    """
    Obtiene los participantes de un pull request específico con sus estados de revisión
    """
    participants_url = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests/{pr_id}/participants"
    participants = get_all_items(participants_url)
    return participants

# Función para obtener información detallada de un pull request
def get_pr_details(workspace, repo_slug, pr_id):
    """
    Obtiene información detallada de un pull request específico
    """
    pr_url = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests/{pr_id}"
    response = requests.get(pr_url, auth=(username, app_password))
    if response.status_code == 200:
        return response.json()
    return None

# Función para obtener información detallada de un pull request incluyendo participants
def get_pr_details_with_participants(workspace, repo_slug, pr_id):
    """
    Obtiene información detallada de un pull request específico incluyendo participants
    """
    pr_url = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests/{pr_id}"
    response = requests.get(pr_url, auth=(username, app_password))
    if response.status_code == 200:
        return response.json()
    return None

# Función para analizar los estados de revisión de los reviewers
def analyze_reviewer_statuses(participants, reviewers_basic):
    """
    Analiza los estados de revisión de los participantes y reviewers
    Retorna conteos de aprobaciones y solicitudes de cambio
    """
    approvals = 0
    changes_requested = 0
    reviewers_info = []
    
    # Crear un diccionario para facilitar la búsqueda
    participants_dict = {}
    for participant in participants:
        if participant.get('user'):
            username = participant['user'].get('nickname') or participant['user'].get('display_name')
            if username:
                participants_dict[username] = participant
    
    # Analizar cada reviewer
    for reviewer in reviewers_basic:
        reviewer_info = {"name": reviewer, "status": "pending"}
        
        # Buscar el reviewer en los participantes
        if reviewer in participants_dict:
            participant = participants_dict[reviewer]
            if participant.get('approved'):
                approvals += 1
                reviewer_info["status"] = "approved"
            elif participant.get('state') == 'changes_requested':
                changes_requested += 1
                reviewer_info["status"] = "changes_requested"
        
        reviewers_info.append(reviewer_info)
    
    return approvals, changes_requested, reviewers_info

# Función para analizar los estados de revisión desde los datos del PR
def analyze_pr_review_statuses(pr_data):
    """
    Analiza los estados de revisión desde los datos completos del PR
    Extrae información de reviewers y participants si está disponible
    """
    approvals = 0
    changes_requested = 0
    reviewers_info = []
    
    # Primero obtener reviewers básicos
    reviewers_basic = []
    if "reviewers" in pr_data and pr_data["reviewers"]:
        for reviewer in pr_data["reviewers"]:
            if "nickname" in reviewer:
                reviewers_basic.append(reviewer["nickname"])
            elif "display_name" in reviewer:
                reviewers_basic.append(reviewer["display_name"])
    
    # Buscar información de participants si está disponible en el PR
    participants = pr_data.get("participants", [])
    
    # Crear diccionario de participantes para facilitar búsqueda
    participants_dict = {}
    for participant in participants:
        if participant.get('user'):
            user_name = participant['user'].get('nickname') or participant['user'].get('display_name')
            if user_name:
                participants_dict[user_name] = participant
    
    # Analizar cada reviewer
    for reviewer in reviewers_basic:
        reviewer_info = {"name": reviewer, "status": "pending"}
        
        # Buscar el reviewer en los participantes
        if reviewer in participants_dict:
            participant = participants_dict[reviewer]
            # Verificar estados de aprobación
            if participant.get('approved'):
                approvals += 1
                reviewer_info["status"] = "approved"
            elif participant.get('state') == 'changes_requested':
                changes_requested += 1
                reviewer_info["status"] = "changes_requested"
        
        reviewers_info.append(reviewer_info)
    
    return approvals, changes_requested, reviewers_info, reviewers_basic

# Función para analizar todos los participants
def analyze_participants_reviewers(pr_data):
    """
    Analiza todos los participants del PR y extrae su estado de aprobación
    - Aprobado: cuando 'approved' es True
    - No aprobado/Pendiente: cuando 'state' es 'changes_requested'
    """
    participants_reviewers = []
    
    # Obtener participants del PR
    participants = pr_data.get("participants", [])
    
    # Procesar todos los participants (sin filtrar por role)
    for participant in participants:
        user_info = participant.get('user', {})
        user_name = user_info.get('display_name') or user_info.get('nickname', 'Usuario Desconocido')
        
        # Verificar el estado según las nuevas reglas
        is_approved = participant.get('approved', False)
        state = participant.get('state', '')
        
        participant_info = {
            "name": user_name,
            "approved": is_approved,
            "state": state
        }
        
        participants_reviewers.append(participant_info)
    
    return participants_reviewers

print("✅ Funciones actualizadas para usar todos los participants con nuevas reglas de estado")

In [ ]:
# Obtener lista de repositorios
url_repos = f"https://api.bitbucket.org/2.0/repositories/{workspace}"

print("🔄 Obteniendo lista de repositorios...")
repositorios_data = get_all_items(url_repos)

if repositorios_data:
    repos_list = []
    for repo in repositorios_data:
        repos_list.append(
            {
                "Nombre": repo.get("name"),
                "URL": repo.get("links", {}).get("html", {}).get("href"),
                "Descripción": repo.get("description", ""),
                "Privado": repo.get("is_private", False),
            }
        )

    repos_df = pd.DataFrame(repos_list)
    repos_df = repos_df.sort_values(by="Nombre")

    # Filtrar repositorios (excluir algunos como en el archivo original)
    repositorios_excluir = ["pgp", "Pruebas_erp", "Inventario", "b2c", "efi"]
    repositorios_activos = [
        repo["Nombre"]
        for repo in repos_list
        if repo["Nombre"] not in repositorios_excluir
    ]

    print(f"✅ Total de repositorios encontrados: {len(repositorios_data)}")
    print(f"✅ Repositorios activos para análisis: {len(repositorios_activos)}")

    # Mostrar algunos repositorios
    display(repos_df.head(10))
else:
    print("❌ No se pudieron obtener los repositorios")

## 🔍 Obtener Pull Requests Abiertos

In [ ]:
# Obtener todos los pull requests abiertos
print("🔄 Obteniendo pull requests abiertos de todos los repositorios...")

pull_requests_abiertos = []
total_repos = len(repositorios_activos)

for i, repo_slug in enumerate(repositorios_activos, 1):
    print(f"   Procesando repositorio {i}/{total_repos}: {repo_slug}")

    # URL para obtener PRs abiertos
    pr_url = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests"

    # Obtener solo PRs abiertos
    params = {"state": "OPEN"}
    prs = get_all_items(pr_url, params=params)

    # Agregar información del repositorio a cada PR
    for pr in prs:
        pr["repository"] = repo_slug

    pull_requests_abiertos.extend(prs)

print(
    f"\n✅ Total de pull requests abiertos encontrados: {len(pull_requests_abiertos)}"
)

if len(pull_requests_abiertos) == 0:
    print("ℹ️  No hay pull requests abiertos en este momento")
else:
    print("🔄 Procesando información adicional de los pull requests...")

In [ ]:
# Procesar información detallada de cada pull request
if len(pull_requests_abiertos) > 0:
    prs_procesados = []

    for i, pr in enumerate(pull_requests_abiertos, 1):
        print(
            f"   Procesando PR {i}/{len(pull_requests_abiertos)}: {pr.get('title', 'Sin título')[:50]}..."
        )

        # Extraer información del autor del PR
        autor = "Desconocido"
        if "author" in pr and pr["author"]:
            if "display_name" in pr["author"]:
                autor = pr["author"]["display_name"]
            elif "nickname" in pr["author"]:
                autor = pr["author"]["nickname"]

        # Obtener comentarios del PR y analizarlos según las nuevas reglas
        try:
            comentarios = get_pr_comments(workspace, pr["repository"], pr["id"])
            comm_pullrequest, comm_codigo = analyze_pr_comments(comentarios, autor)
        except Exception as e:
            print(f"      ⚠️  Error obteniendo comentarios: {str(e)}")
            comm_pullrequest = 0
            comm_codigo = 0

        # Obtener participants reviewers del PR
        participants_reviewers = []
        
        try:
            # Obtener datos completos del PR desde el endpoint principal
            pr_details = get_pr_details_with_participants(workspace, pr["repository"], pr["id"])
            if pr_details:
                participants_reviewers = analyze_participants_reviewers(pr_details)
        except Exception as e:
            print(f"      ⚠️  Error obteniendo participants del PR: {str(e)}")

        # Crear string para mostrar reviewers con estados de aprobación
        reviewers_display = []
        for participant in participants_reviewers:
            name = participant["name"]
            approved = participant["approved"]
            state = participant["state"]
            
            # Acortar el nombre si es necesario
            name_short = acortar_nombre(name)
            
            if approved and state == "approved":
                reviewers_display.append(f"{name_short}✅")
            elif state == "changes_requested":
                reviewers_display.append(f"{name_short}❌")

        # Calcular días abierto
        created_on = pd.to_datetime(pr["created_on"])
        now = pd.Timestamp.now(tz="UTC")
        dias_abierto = (now - created_on).days

        # Crear registro procesado (sin Review_Status)
        pr_procesado = {
            "ID": pr["id"],
            "Título": pr.get("title", "Sin título"),
            "Repositorio": pr["repository"],
            "Autor": autor,
            "Reviewers": ", ".join(reviewers_display) if reviewers_display else "--",
            "Cantidad_Reviewers": len(participants_reviewers),
            "Devoluciones": comm_pullrequest,
            "Comm_Codigo": comm_codigo,
            "Fecha_Creacion": created_on,  # Mantener como objeto datetime
            "Ultima_Actualizacion": pd.to_datetime(pr["updated_on"]),  # Mantener como objeto datetime
            "Dias_Abierto": dias_abierto,
            "Descripcion": (pr.get("description", "Sin descripción")[:100] + "..." if pr.get("description", "") else "Sin descripción"),
            "URL": pr.get("links", {}).get("html", {}).get("href", ""),
            "Rama_Origen": pr.get("source", {}).get("branch", {}).get("name", "Desconocida"),
            "Rama_Destino": pr.get("destination", {}).get("branch", {}).get("name", "Desconocida"),
        }

        prs_procesados.append(pr_procesado)

    # Crear DataFrame con los datos procesados
    df_prs_abiertos = pd.DataFrame(prs_procesados)

    # Ordenar por días abierto (descendente) para mostrar los más antiguos primero
    df_prs_abiertos = df_prs_abiertos.sort_values("Dias_Abierto", ascending=False)

    print(
        f"\n✅ Procesamiento completado. {len(df_prs_abiertos)} pull requests procesados"
    )
else:
    df_prs_abiertos = pd.DataFrame()

## 📊 Reportes y Análisis

In [ ]:
# Resumen ejecutivo
if not df_prs_abiertos.empty:
    print("=" * 60)
    print("📈 RESUMEN EJECUTIVO - PULL REQUESTS ABIERTOS")
    print("=" * 60)

    total_prs = len(df_prs_abiertos)
    prs_con_reviewers = len(df_prs_abiertos[df_prs_abiertos["Cantidad_Reviewers"] > 0])
    prs_sin_reviewers = total_prs - prs_con_reviewers

    prs_con_comentarios = len(df_prs_abiertos[df_prs_abiertos["Devoluciones"] > 0])
    total_comentarios = df_prs_abiertos["Devoluciones"].sum()
    total_comentarios_codigo = df_prs_abiertos["Comm_Codigo"].sum()

    promedio_dias_abierto = df_prs_abiertos["Dias_Abierto"].mean()
    max_dias_abierto = df_prs_abiertos["Dias_Abierto"].max()

    print(f"🔢 Total de Pull Requests Abiertos: {total_prs}")
    print(
        f"📝 PRs con Reviewers Asignados: {prs_con_reviewers} ({prs_con_reviewers / total_prs * 100:.1f}%)"
    )
    print(
        f"⚠️  PRs sin Reviewers: {prs_sin_reviewers} ({prs_sin_reviewers / total_prs * 100:.1f}%)"
    )
    print(f"💬 PRs con Comentarios de Terceros: {prs_con_comentarios}")
    print(f"📊 Total de Comentarios de Terceros: {total_comentarios}")
    print(f"👨‍💻 Comentarios de Código (inline): {total_comentarios_codigo}")
    print(f"⏱️  Promedio de Días Abierto: {promedio_dias_abierto:.1f} días")
    print(f"🚨 PR más Antiguo: {max_dias_abierto} días")

    # Repositorios con más PRs abiertos
    repos_prs = df_prs_abiertos["Repositorio"].value_counts()
    print(f"\n🏆 Repositorios con más PRs abiertos:")
    for i, (repo, count) in enumerate(repos_prs.head(5).items(), 1):
        print(f"   {i}. {repo}: {count} PRs")

    # Autores con más PRs abiertos
    autores_prs = df_prs_abiertos["Autor"].value_counts()
    print(f"\n👥 Autores con más PRs abiertos:")
    for i, (autor, count) in enumerate(autores_prs.head(5).items(), 1):
        print(f"   {i}. {autor}: {count} PRs")

    print("=" * 60)
else:
    print("ℹ️  No hay pull requests abiertos para mostrar en el resumen")

In [ ]:
# Mostrar tabla detallada de Pull Requests
if not df_prs_abiertos.empty:
    print("\n📋 DETALLE DE PULL REQUESTS ABIERTOS")
    print("=" * 80)

    # Crear una versión de la tabla optimizada para visualización (sin Review_Status)
    df_display = df_prs_abiertos[
        [
            "ID",
            "Repositorio",
            "Título",
            "Autor",
            "Reviewers",
            "Devoluciones",
            "Comm_Codigo",
            "Dias_Abierto",
            "Rama_Origen",
            "Fecha_Creacion",
            "Ultima_Actualizacion",
        ]
    ].copy()

    # Ordenar primero por días abierto (usando la columna numérica original)
    df_display = df_display.sort_values("Dias_Abierto", ascending=False)

    # Convertir fechas a formato "para humanos"
    df_display["Creado"] = df_display["Fecha_Creacion"].apply(fecha_para_humanos)
    df_display["Actualizado"] = df_display["Ultima_Actualizacion"].apply(
        fecha_para_humanos
    )

    # Eliminar las columnas de fecha originales
    df_display = df_display.drop(columns=["Fecha_Creacion", "Ultima_Actualizacion"])

    # Modificar la columna Comm_Codigo para incluir el indicador visual
    df_display["Comm_Codigo"] = df_display["Comm_Codigo"].apply(
        lambda count: f"{count} {'🔴' if count >= 10 else '🟡' if count >= 4 else '🟢'}"
    )

    # Agregar ícono de alerta para PRs con más de 7 días abiertos
    df_display["Dias_Abierto"] = df_display["Dias_Abierto"].apply(
        lambda dias: f"🐌 {dias}" if dias >= 7 else str(dias)
    )

    # Renombrar columnas para mayor claridad
    df_display = df_display.rename(columns={"Rama_Origen": "Branch"})

    # Reordenar columnas finales (sin Review Status)
    df_display = df_display[
        [
            "ID",
            "Repositorio",
            "Título",
            "Autor",
            "Branch",
            "Reviewers",
            "Devoluciones",
            "Comm_Codigo",
            "Dias_Abierto",
            "Creado",
            "Actualizado",
        ]
    ]

    # Configurar pandas para mostrar todas las columnas
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", None)
    pd.set_option("display.max_colwidth", 50)

    # Mostrar la tabla
    display(df_display)

    print(f"\n📌 Mostrando {len(df_display)} pull requests abiertos")
    print("💡 Tip: Los PRs están ordenados por días abierto (más antiguos primero)")
    print("⚠️  Los PRs con más de 7 días muestran un ícono de alerta")
    print("👥 Reviewers: ✅ Aprobado | ❌ No aprobado/Pendiente | -- Sin reviewers")
else:
    print("🎉 ¡Excelente! No hay pull requests abiertos en este momento")

In [ ]:
# Visualizaciones
if not df_prs_abiertos.empty:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # 1. PRs por repositorio
    repos_count = df_prs_abiertos["Repositorio"].value_counts().head(10)
    axes[0, 0].bar(range(len(repos_count)), repos_count.values, color="skyblue")
    axes[0, 0].set_title("PRs Abiertos por Repositorio (Top 10)")
    axes[0, 0].set_xlabel("Repositorio")
    axes[0, 0].set_ylabel("Cantidad de PRs")
    axes[0, 0].set_xticks(range(len(repos_count)))
    axes[0, 0].set_xticklabels(repos_count.index, rotation=45, ha="right")

    # 2. Distribución de días abierto
    axes[0, 1].hist(
        df_prs_abiertos["Dias_Abierto"], bins=20, color="lightcoral", alpha=0.7
    )
    axes[0, 1].set_title("Distribución de Días Abierto")
    axes[0, 1].set_xlabel("Días Abierto")
    axes[0, 1].set_ylabel("Cantidad de PRs")
    axes[0, 1].axvline(
        df_prs_abiertos["Dias_Abierto"].mean(),
        color="red",
        linestyle="--",
        label=f"Promedio: {df_prs_abiertos['Dias_Abierto'].mean():.1f} días",
    )
    axes[0, 1].legend()

    # 3. PRs con y sin reviewers
    reviewers_data = (
        df_prs_abiertos["Cantidad_Reviewers"]
        .apply(lambda x: "Con Reviewers" if x > 0 else "Sin Reviewers")
        .value_counts()
    )
    axes[1, 0].pie(
        reviewers_data.values,
        labels=reviewers_data.index,
        autopct="%1.1f%%",
        colors=["lightgreen", "lightcoral"],
    )
    axes[1, 0].set_title("PRs: Con vs Sin Reviewers")

    # 4. Comentarios de Pull Request vs Comentarios de Código
    prs_con_comentarios = df_prs_abiertos[df_prs_abiertos["Devoluciones"] > 0]
    if not prs_con_comentarios.empty:
        axes[1, 1].scatter(
            prs_con_comentarios["Devoluciones"],
            prs_con_comentarios["Comm_Codigo"],
            alpha=0.6,
            color="orange",
        )
        axes[1, 1].set_title("Comentarios Pull Request vs Comentarios Código")
        axes[1, 1].set_xlabel("Comentarios de Pull Request (Terceros)")
        axes[1, 1].set_ylabel("Comentarios de Código (Inline)")

        # Línea diagonal para mostrar cuando todos los comentarios son de código
        max_comments = max(
            prs_con_comentarios["Devoluciones"].max(),
            prs_con_comentarios["Comm_Codigo"].max(),
        )
        axes[1, 1].plot(
            [0, max_comments],
            [0, max_comments],
            "r--",
            alpha=0.5,
            label="Todos son comentarios de código",
        )
        axes[1, 1].legend()
    else:
        axes[1, 1].text(
            0.5,
            0.5,
            "No hay PRs\ncon comentarios",
            horizontalalignment="center",
            verticalalignment="center",
            transform=axes[1, 1].transAxes,
            fontsize=12,
        )
        axes[1, 1].set_title("Comentarios Pull Request vs Comentarios Código")

    plt.tight_layout()
    plt.show()
else:
    print("📊 No hay datos para generar gráficos")

In [ ]:
# Alertas y recomendaciones para gestión
if not df_prs_abiertos.empty:
    print("\n🚨 ALERTAS Y RECOMENDACIONES")
    print("=" * 60)

    # PRs antiguos (más de 7 días)
    prs_antiguos = df_prs_abiertos[df_prs_abiertos["Dias_Abierto"] > 7]
    if not prs_antiguos.empty:
        print(f"⏰ PRs ANTIGUOS (más de 7 días): {len(prs_antiguos)}")
        for _, pr in prs_antiguos.head(5).iterrows():
            print(
                f"   • {pr['Repositorio']}: '{pr['Título'][:40]}...' - {pr['Dias_Abierto']} días"
            )
        if len(prs_antiguos) > 5:
            print(f"   ... y {len(prs_antiguos) - 5} más")

    # PRs sin reviewers
    prs_sin_reviewers = df_prs_abiertos[df_prs_abiertos["Cantidad_Reviewers"] == 0]
    if not prs_sin_reviewers.empty:
        print(f"\n👥 PRs SIN REVIEWERS: {len(prs_sin_reviewers)}")
        for _, pr in prs_sin_reviewers.head(5).iterrows():
            print(f"   • {pr['Repositorio']}: '{pr['Título'][:40]}...' - {pr['Autor']}")
        if len(prs_sin_reviewers) > 5:
            print(f"   ... y {len(prs_sin_reviewers) - 5} más")

    # PRs con muchos comentarios de código
    prs_con_comentarios_codigo = df_prs_abiertos[
        df_prs_abiertos["Comm_Codigo"] >= 2
    ]
    if not prs_con_comentarios_codigo.empty:
        print(
            f"\n👨‍💻 PRs CON MUCHOS COMENTARIOS DE CÓDIGO (2+): {len(prs_con_comentarios_codigo)}"
        )
        for _, pr in prs_con_comentarios_codigo.head(5).iterrows():
            porcentaje_codigo = (pr["Comm_Codigo"] / pr["Devoluciones"] * 100) if pr["Devoluciones"] > 0 else 0
            print(
                f"   • {pr['Repositorio']}: '{pr['Título'][:40]}...' - {pr['Comm_Codigo']} comentarios código ({porcentaje_codigo:.0f}%)"
            )
        if len(prs_con_comentarios_codigo) > 5:
            print(f"   ... y {len(prs_con_comentarios_codigo) - 5} más")

    print(f"\n💡 RECOMENDACIONES:")
    print(f"   • Revisar PRs antiguos para acelerar el proceso de merge")
    print(f"   • Asignar reviewers a PRs que no los tienen")
    print(f"   • Atender comentarios de código (inline) que requieren cambios")
    print(f"   • Priorizar PRs con más comentarios críticos de código")
    print(f"   • Considerar establecer SLAs para tiempo máximo de PRs abiertos")

    print("=" * 60)
else:
    print("✅ ¡Excelente! No hay pull requests abiertos que requieran atención")